In [1]:
from pathlib import Path
from astropy.io import fits
import pandas as pd
import numpy as np
import collections
import os
import matplotlib.pyplot as plt
%matplotlib widget

datapath = Path("/data01/home/baroncelli/phd/DATA")
os.environ["DATA"] = str(datapath)

from RTAscience.cfg.Config import Config
from RTAscience.lib.RTAUtils import get_pointing
from rtapipe.pyscripts.phlists_to_photometry_plot import Photometry, PhotometrySinglePlot, PhotometrySubPlots


Bad key "text.kerning_factor" on line 4 in
/data01/home/baroncelli/.conda/envs/bphd/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
def getInput(dataDir, index):
    if index == 0:
        index = 1
    simFolder = datapath.joinpath("obs",dataDir)
    cfg = Config(simFolder.joinpath("config.yaml"))
    print(cfg.get("emin"))
    print(cfg.get("emax"))
    runid = cfg.get('runid')
    template =  os.path.join(datapath, f'templates/{runid}.fits')
    pointing = get_pointing(template)
    if cfg.get("simtype") == 'bkg':
        inputFitsFile = simFolder.joinpath("backgrounds",f"bkg{str(index).zfill(6)}.fits")
    else:
        inputFitsFile = simFolder.joinpath(runid,f"ebl{str(index).zfill(6)}.fits")    
    return inputFitsFile, cfg, pointing

In [3]:
bkgsrc      = "obs_st_grb_tr_10_os_1800_emin_0.03_emax_0.15_roi_2.5"
bkgonly     = "obs_st_bkg_tr_10_os_0_emin_0.03_emax_0.15_roi_2.5"

In [4]:
inputFile_srcbkg_0, cfg_srcbkg_0, pointing_srcbkg_0 = getInput(bkgsrc, 1)
print("inputFile_srcbkg_0: ",inputFile_srcbkg_0)

0.03
0.15
inputFile_srcbkg_0:  /data01/home/baroncelli/phd/DATA/obs/obs_st_grb_tr_10_os_1800_emin_0.03_emax_0.15_roi_2.5/run0406_ID000126/ebl000001.fits


In [5]:
inputFile_onlybkg_0, cfg_onlybkg_0, pointing_onlybkg_0 = getInput(bkgonly, 1)
print("inputFile_onlybkg_0: ",inputFile_onlybkg_0)

0.03
0.15
inputFile_onlybkg_0:  /data01/home/baroncelli/phd/DATA/obs/obs_st_bkg_tr_10_os_0_emin_0.03_emax_0.15_roi_2.5/backgrounds/bkg000001.fits


In [6]:
assert pointing_srcbkg_0 == pointing_onlybkg_0
assert inputFile_srcbkg_0 != inputFile_onlybkg_0
for key, val in cfg_srcbkg_0.cfg.items():
    for subkey, subval in val.items():
        if subval!=cfg_onlybkg_0.cfg[key][subkey]:
            print(key,subkey,subval,cfg_onlybkg_0.cfg[key][subkey])


setup simtype grb bkg
simulation tobs 3600 86400
simulation onset 1800 0


## The photometry class

In [7]:
photometry = Photometry()

## Input parameters

### Two simulation files: bkg only and bkg + source


#### Template: run0406_ID000126
 Same template ==> same parameters, except:
* input file
* simtype
* onset
* obs_dir

In [44]:
outdir = os.path.join(os.environ["HOME"], "notebook_output")
csvdir = os.path.join(outdir, "csv")
pngdir = os.path.join(outdir, "png")
! mkdir -p $csvdir                          
! mkdir -p $pngdir                          
! ls $outdir

bkgsrc	csv  png  src


In [9]:
# background + source 
bkg_src_conf = {
    'input_file': inputFile_srcbkg_0,
    'output_dir': outdir
    'simtype' : cfg_srcbkg_0.get('simtype'),
    'runid' : cfg_srcbkg_0.get('runid'),
    't_window_start': 0,
    't_window_stop': cfg_srcbkg_0.get('tobs'),
    'e_window_start': cfg_srcbkg_0.get('emin'),
    'e_window_stop': cfg_srcbkg_0.get('emax')
}

# background only
bkg_only_conf = {
    'input_file': inputFile_onlybkg_0,
    'output_dir': outdir,
    'simtype' : cfg_onlybkg_0.get('simtype'),
    'runid' : cfg_onlybkg_0.get('runid'),
    't_window_start': 0,
    't_window_stop': 3600, # cfg_onlybkg_0.get('tobs'),
    'e_window_start': cfg_onlybkg_0.get('emin'),
    'e_window_stop': cfg_onlybkg_0.get('emax'),
}

## Photometry parameters

In [10]:
photometry_params = {
    
    # integration parameters for time
    't_window_size': 25,
    't_window_step': 25,
    
    # integration parameters for energy    
    'e_window_size': 0.001,
    'e_window_step': 0.001,    

    # Parameters that can change too    
    'pointing' : pointing_srcbkg_0,
    'region_radius': 0.5,
    
    # Other settings
    'plot' : 1,
    'onset' : 1800,
    'override': 1
}

## Generating some data

### Integrate on time 

In [11]:
csv_data_t_bkg_src = photometry.integrate(photometry_params, bkg_src_conf, integration="t")
#csv_data_t_bkg_src = "/data01/home/baroncelli/phd/DATA/obs/obs_st_bkg_tr_10_os_0_emin_0.03_emax_0.15_roi_2.5/backgrounds/bkg000001.fits"
pd.read_csv(csv_data_t_bkg_src).head(3)

  3%|▎         | 4/144 [00:00<00:03, 36.45it/s]

input_file:  /data01/home/baroncelli/phd/DATA/obs/obs_st_grb_tr_10_os_1800_emin_0.03_emax_0.15_roi_2.5/run0406_ID000126/ebl000001.fits
input_file_basename:  ebl000001.fits


100%|██████████| 144/144 [00:01<00:00, 77.25it/s]

Produced:  /data01/home/baroncelli/notebook_output/src/ebl000001_qztncyzsbhmlwsw.csv


,VALMIN,VALMAX,VALCENTER,COUNTS,ERROR
0,0,25,12.5,168,12.961481
1,25,50,37.5,177,13.304135
2,50,75,62.5,165,12.845233


In [12]:
csv_data_t_bkg_only = photometry.integrate(photometry_params, bkg_only_conf, integration="t")
# csv_data_t_bkg_only = "/data01/home/baroncelli/phd/DATA/obs/obs_st_grb_tr_10_os_1800_emin_0.03_emax_0.15_roi_2.5/run0406_ID000126/ebl000001.fits"
print(pd.read_csv(csv_data_t_bkg_only))

  0%|          | 0/144 [00:00<?, ?it/s]

input_file:  /data01/home/baroncelli/phd/DATA/obs/obs_st_bkg_tr_10_os_0_emin_0.03_emax_0.15_roi_2.5/backgrounds/bkg000001.fits
input_file_basename:  bkg000001.fits


100%|██████████| 144/144 [00:47<00:00,  3.02it/s]

Produced:  ./notebook_output/bkgsrc/bkg000001_yhdzvqzerngxoqm.csv
     VALMIN  VALMAX  VALCENTER  COUNTS      ERROR
0         0      25       12.5     155  12.449900
1        25      50       37.5     184  13.564660
2        50      75       62.5     181  13.453624
3        75     100       87.5     160  12.649111
4       100     125      112.5     183  13.527749
..      ...     ...        ...     ...        ...
139    3475    3500     3487.5     144  12.000000
140    3500    3525     3512.5     158  12.569805
141    3525    3550     3537.5     156  12.489996
142    3550    3575     3562.5     159  12.609520
143    3575    3600     3587.5     166  12.884099

[144 rows x 5 columns]


### Integrate on energy

In [13]:
csv_data_e_bkg_src = photometry.integrate(photometry_params, bkg_src_conf, integration="e")

  3%|▎         | 4/119 [00:00<00:03, 37.38it/s]

input_file:  /data01/home/baroncelli/phd/DATA/obs/obs_st_grb_tr_10_os_1800_emin_0.03_emax_0.15_roi_2.5/run0406_ID000126/ebl000001.fits
input_file_basename:  ebl000001.fits


100%|██████████| 119/119 [00:01<00:00, 76.52it/s]

Produced:  /data01/home/baroncelli/notebook_output/src/ebl000001_ezqqyuhjncyglyh.csv


In [14]:
csv_data_e_bkg_only = photometry.integrate(photometry_params, bkg_only_conf, integration="e")

  0%|          | 0/119 [00:00<?, ?it/s]

input_file:  /data01/home/baroncelli/phd/DATA/obs/obs_st_bkg_tr_10_os_0_emin_0.03_emax_0.15_roi_2.5/backgrounds/bkg000001.fits
input_file_basename:  bkg000001.fits


100%|██████████| 119/119 [00:39<00:00,  2.98it/s]

Produced:  ./notebook_output/bkgsrc/bkg000001_xcblpzoqkmesqzd.csv


## Plotting the data

### Time integration

In [15]:
singleplot_t = PhotometrySinglePlot()
_ = singleplot_t.addData(csv_data_t_bkg_src, photometry_params, ["bkg+src"], "t")
_ = singleplot_t.addData(csv_data_t_bkg_only, photometry_params, ["bkg only"], "t")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
multiplot_t = PhotometrySubPlots()
_ = multiplot_t.addData(csv_data_t_bkg_src, photometry_params, ["bkg+src"], "t")
_ = multiplot_t.addData(csv_data_t_bkg_only, photometry_params, ["bkg only"], "t")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Energy integration

In [17]:
singleplot_e = PhotometrySinglePlot()
singleplot_e.addData(csv_data_e_bkg_src, photometry_params, ["bkg+src"], "e")
singleplot_e.addData(csv_data_e_bkg_only, photometry_params, ["bkg only"], "e")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
multiplot_e = PhotometrySubPlots()
_ = multiplot_e.addData(csv_data_e_bkg_src, photometry_params, ["bkg+src"], "e")
_ = multiplot_e.addData(csv_data_e_bkg_only, photometry_params, ["bkg only"], "e")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Integration on time with multiple window sizes and background baseline

In [19]:
# first plot: background + source on multiple window sizes
singleplot_bgksrc = PhotometrySinglePlot()

# second plot: background only on multiple window sizes
singleplot_bgk = PhotometrySinglePlot()

# third plot: background + source on multiple window sizes (separate subplots with background baseline)
multiplot_bgksrc = PhotometrySubPlots()

photometry_params_tmp = photometry_params.copy()

for ws in [10, 25, 50, 75]:

    # source+background
    photometry_params_tmp["t_window_size"] = ws
    photometry_params_tmp["t_window_step"] = ws 
    csv_data_t_bkg_src = photometry.integrate(photometry_params_tmp, bkg_src_conf, integration="t")
    
    
    # background only baseline
    photometry_params_tmp["t_window_size"] = ws
    photometry_params_tmp["t_window_step"] = ws
    csv_data_t_bkg = photometry.integrate(photometry_params_tmp, bkg_only_conf, integration="t")
    

    _ = singleplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, ["t_window_size","t_window_step"], "t")

    _ = singleplot_bgk.addData(csv_data_t_bkg, photometry_params_tmp, ["t_window_size","t_window_step"], "t")
    
    
    _ = multiplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, ["t_window_size","t_window_step"], "t")
    _ = multiplot_bgksrc.addData(csv_data_t_bkg, photometry_params_tmp, ["t_window_size","t_window_step"], "t", as_baseline=True)
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

  2%|▏         | 6/360 [00:00<00:06, 52.19it/s]

input_file:  /data01/home/baroncelli/phd/DATA/obs/obs_st_grb_tr_10_os_1800_emin_0.03_emax_0.15_roi_2.5/run0406_ID000126/ebl000001.fits
input_file_basename:  ebl000001.fits


...

Produced:  ./notebook_output/bkgsrc/bkg000001_dgdqnpynqhrobpv.csv
input_file:  /data01/home/baroncelli/phd/DATA/obs/obs_st_grb_tr_10_os_1800_emin_0.03_emax_0.15_roi_2.5/run0406_ID000126/ebl000001.fits
input_file_basename:  ebl000001.fits


  0%|          | 0/48 [00:00<?, ?it/s]

Produced:  /data01/home/baroncelli/notebook_output/src/ebl000001_elpvvgputhzvsel.csv
input_file:  /data01/home/baroncelli/phd/DATA/obs/obs_st_bkg_tr_10_os_0_emin_0.03_emax_0.15_roi_2.5/backgrounds/bkg000001.fits
input_file_basename:  bkg000001.fits


100%|██████████| 48/48 [00:15<00:00,  3.18it/s]

Produced:  ./notebook_output/bkgsrc/bkg000001_iwzagnlqkkbhezs.csv


In [39]:
singleplot_bgksrc.save(os.path.join(outdir, "png/singleplot_t_ws_bgksrc"))
singleplot_bgksrc.destroy()
singleplot_bgk.save(os.path.join(outdir, "png/singleplot_t_ws_bgkonly"))
singleplot_bgk.destroy()
multiplot_bgksrc.save(os.path.join(outdir, "png/multiplot_t_ws_bgksrc"))
multiplot_bgksrc.destroy()

Produced:  /data01/home/baroncelli/notebook_output/png/singleplot_t_ws_bgksrc.png
Produced:  /data01/home/baroncelli/notebook_output/png/singleplot_t_ws_bgkonly.png
Produced:  /data01/home/baroncelli/notebook_output/png/multiplot_t_ws_bgksrc.png


## Integration on energy with multiple window sizes and background baseline

In [40]:
# first plot: background + source on multiple window sizes
singleplot_bgksrc = PhotometrySinglePlot()

# second plot: background only on multiple window sizes
singleplot_bgk = PhotometrySinglePlot()

# third plot: background + source on multiple window sizes (separate subplots with background baseline)
multiplot_bgksrc = PhotometrySubPlots()

photometry_params_tmp = photometry_params.copy()

for ee in [0.001, 0.01, 0.02, 0.03]:

    # source+background
    photometry_params_tmp["e_window_size"] = ee
    photometry_params_tmp["e_window_step"] = ee 
    csv_data_t_bkg_src = photometry.integrate(photometry_params_tmp, bkg_src_conf, integration="e")
    
    
    # background only baseline
    photometry_params_tmp["e_window_size"] = ee
    photometry_params_tmp["e_window_step"] = ee
    csv_data_t_bkg = photometry.integrate(photometry_params_tmp, bkg_only_conf, integration="e")
    

    _ = singleplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, ["e_window_size","e_window_step"], "e")

    _ = singleplot_bgk.addData(csv_data_t_bkg, photometry_params_tmp, ["e_window_size","e_window_step"], "e")
    
    
    _ = multiplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, ["t_window_size","t_window_step"], "e")
    _ = multiplot_bgksrc.addData(csv_data_t_bkg, photometry_params_tmp, ["t_window_size","t_window_step"], "e", as_baseline=True)
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

  5%|▌         | 6/119 [00:00<00:02, 51.91it/s]

input_file:  /data01/home/baroncelli/phd/DATA/obs/obs_st_grb_tr_10_os_1800_emin_0.03_emax_0.15_roi_2.5/run0406_ID000126/ebl000001.fits
input_file_basename:  ebl000001.fits


...

Produced:  ./notebook_output/bkgsrc/bkg000001_xyjzuilntwyvkby.csv
input_file:  /data01/home/baroncelli/phd/DATA/obs/obs_st_grb_tr_10_os_1800_emin_0.03_emax_0.15_roi_2.5/run0406_ID000126/ebl000001.fits
input_file_basename:  ebl000001.fits
Produced:  /data01/home/baroncelli/notebook_output/src/ebl000001_kewprohwortwowd.csv
input_file:  /data01/home/baroncelli/phd/DATA/obs/obs_st_bkg_tr_10_os_0_emin_0.03_emax_0.15_roi_2.5/backgrounds/bkg000001.fits
input_file_basename:  bkg000001.fits



  0%|          | 0/4 [00:00<?, ?it/s]

Produced:  ./notebook_output/bkgsrc/bkg000001_tkxppkxljhvzvxr.csv
input_file:  /data01/home/baroncelli/phd/DATA/obs/obs_st_grb_tr_10_os_1800_emin_0.03_emax_0.15_roi_2.5/run0406_ID000126/ebl000001.fits
input_file_basename:  ebl000001.fits
Produced:  /data01/home/baroncelli/notebook_output/src/ebl000001_nakedhahnnibmri.csv
input_file:  /data01/home/baroncelli/phd/DATA/obs/obs_st_bkg_tr_10_os_0_emin_0.03_emax_0.15_roi_2.5/backgrounds/bkg000001.fits
input_file_basename:  bkg000001.fits


100%|██████████| 4/4 [00:01<00:00,  2.43it/s]

Produced:  ./notebook_output/bkgsrc/bkg000001_pxjrmbogrkfnass.csv


In [41]:
singleplot_bgksrc.save(os.path.join(outdir, "png/singleplot_e_ws_bgksrc"))
singleplot_bgksrc.destroy()
singleplot_bgk.save(os.path.join(outdir, "png/singleplot_e_ws_bgkonly"))
singleplot_bgk.destroy()
multiplot_bgksrc.save(os.path.join(outdir, "png/multiplot_e_ws_bgksrc"))
multiplot_bgksrc.destroy()

Produced:  /data01/home/baroncelli/notebook_output/png/singleplot_e_ws_bgksrc.png
Produced:  /data01/home/baroncelli/notebook_output/png/singleplot_e_ws_bgkonly.png
Produced:  /data01/home/baroncelli/notebook_output/png/multiplot_e_ws_bgksrc.png


## Integration on time with multiple region radius values and background baseline
 

In [43]:
# first plot: background + source on multiple window sizes
singleplot_bgksrc = PhotometrySinglePlot()

# second plot: background only on multiple window sizes
singleplot_bgk = PhotometrySinglePlot()

# third plot: background + source on multiple window sizes (separate subplots with background baseline)
multiplot_bgksrc = PhotometrySubPlots()

photometry_params_tmp = photometry_params.copy()

for rr in [0.1, 1.0, 2.0, 3.0]:

    # source+background
    photometry_params_tmp["region_radius"] = rr
    photometry_params_tmp["region_radius"] = rr 
    csv_data_t_bkg_src = photometry.integrate(photometry_params_tmp, bkg_src_conf, integration="t")
    
    
    # background only baseline
    photometry_params_tmp["region_radius"] = rr
    photometry_params_tmp["region_radius"] = rr
    csv_data_t_bkg = photometry.integrate(photometry_params_tmp, bkg_only_conf, integration="t")
    

    _ = singleplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, ["region_radius"], "t")

    _ = singleplot_bgk.addData(csv_data_t_bkg, photometry_params_tmp, ["region_radius"], "t")
    
    
    _ = multiplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, ["region_radius"], "t")
    _ = multiplot_bgksrc.addData(csv_data_t_bkg, photometry_params_tmp, ["region_radius"], "t", as_baseline=True)
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

  3%|▎         | 5/144 [00:00<00:02, 48.21it/s]

input_file:  /data01/home/baroncelli/phd/DATA/obs/obs_st_grb_tr_10_os_1800_emin_0.03_emax_0.15_roi_2.5/run0406_ID000126/ebl000001.fits
input_file_basename:  ebl000001.fits


100%|██████████| 144/144 [00:01<00:00, 76.90it/s]

Produced:  /data01/home/baroncelli/notebook_output/src/ebl000001_xlxqdsbrqawzfhi.csv
input_file:  /data01/home/baroncelli/phd/DATA/obs/obs_st_bkg_tr_10_os_0_emin_0.03_emax_0.15_roi_2.5/backgrounds/bkg000001.fits
input_file_basename:  bkg000001.fits


FileNotFoundError: [Errno 2] No such file or directory: './notebook_output/bkgsrc/bkg000001_cnghvwoxkzpysef.csv'

In [ ]:
singleplot_bgksrc.save(os.path.join(outdir, "png/singleplot_t_rr_bgksrc"))
singleplot_bgksrc.destroy()
singleplot_bgk.save(os.path.join(outdir, "png/singleplot_t_rr_bgkonly"))
singleplot_bgk.destroy()
multiplot_bgksrc.save(os.path.join(outdir, "png/multiplot_t_rr_bgksrc"))
multiplot_bgksrc.destroy()

## Integration on time with multiple pointing values and baseline on original pointing


In [ ]:
# first plot: background + source on multiple window sizes
singleplot_bgksrc = PhotometrySinglePlot()

# second plot: background only on multiple window sizes
singleplot_bgk = PhotometrySinglePlot()

# third plot: background + source on multiple window sizes (separate subplots with background baseline)
multiplot_bgksrc = PhotometrySubPlots()

photometry_params_tmp = photometry_params.copy()

pointing_offset = 0.5
pointings = [
    (pointing[0]+pointing_offset,pointing[1]), 
    (pointing[0],pointing[1]+pointing_offset), 
    (pointing[0]-pointing_offset,pointing[1]), 
    (pointing[0],pointing[1]-pointing_offset)
] 

csv_data_t_orig_point = photometry.integrate(photometry_params_tmp, bkg_src_conf, integration="t")

for pp in pointings:

    # source+background
    photometry_params_tmp["pointing"] = pp
    photometry_params_tmp["pointing"] = pp 
    csv_data_t_bkg_src = photometry.integrate(photometry_params_tmp, bkg_src_conf, integration="t")
    
    
    # background only baseline
    photometry_params_tmp["pointing"] = pp
    photometry_params_tmp["pointing"] = pp
    csv_data_t_bkg = photometry.integrate(photometry_params_tmp, bkg_only_conf, integration="t")
    

    _ = singleplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, ["pointing"], "t")

    _ = singleplot_bgk.addData(csv_data_t_bkg, photometry_params_tmp, ["pointing"], "t")
    
    
    _ = multiplot_bgksrc.addData(csv_data_t_bkg_src, photometry_params_tmp, ["pointing"], "t")
    _ = multiplot_bgksrc.addData(csv_data_t_orig_point, photometry_params, ["pointing"], "t", as_baseline=True)

In [ ]:
singleplot_bgksrc.save(os.path.join(outdir, "png/singleplot_t_p_bgksrc"))
singleplot_bgksrc.destroy()
singleplot_bgk.save(os.path.join(outdir, "png/singleplot_t_p_bgkonly"))
singleplot_bgk.destroy()
multiplot_bgksrc.save(os.path.join(outdir, "png/multiplot_t_p_bgksrc"))
multiplot_bgksrc.destroy()